In [1]:
import pandas as pd
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
import re
# csv -> json

submission = pd.read_csv('/data/ephemeral/home/whth/level2-objectdetection-cv-16/train_pred_for_eda.csv')

coco_submission = []

def extract_image_id(image_path):
    image_id = re.findall(r'\d+', image_path)[0]
    return int(image_id)

for idx, row in submission.iterrows():
    image_id = extract_image_id(row['image_id'])
    predictions = row['PredictionString'].split(' ')
    
    predictions = [pred for pred in predictions if pred.strip()]

    for i in range(0, len(predictions), 6):
        category_id = int(predictions[i])
        score = float(predictions[i + 1])
        bbox = [float(predictions[i + 2]), float(predictions[i + 3]), 
                float(predictions[i + 4]) - float(predictions[i + 2]), 
                float(predictions[i + 5]) - float(predictions[i + 3])] 

        prediction = {
            "image_id": image_id,
            "category_id": category_id,
            "bbox": bbox,
            "score": score
        }
        coco_submission.append(prediction)

with open('/data/ephemeral/home/whth/level2-objectdetection-cv-16/coco_submission.json', 'w') as f:
    json.dump(coco_submission, f, indent=4)



In [3]:
coco_gt = COCO('/data/ephemeral/home/whth/level2-objectdetection-cv-16/dataset/train.json') # gt데이터
coco_dt = coco_gt.loadRes('/data/ephemeral/home/whth/level2-objectdetection-cv-16/coco_submission.json') # json으로 바꾼거경로따오기
# coco_gt, coco_pred 선언법 걍 이게 맞다 ㅇㅋ

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.71s)
creating index...
index created!


In [4]:
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=16.14s).
Accumulating evaluation results...
DONE (t=2.20s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.648
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.828
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.700
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.033
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.727
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.368
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.668
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.709
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.098
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=10

In [5]:
metrics = {
    "category": [],
    "mAP": [],
    "mAP_50": [],
    "mAP_75": [],
    "mAP_s": [],
    "mAP_m": [],
    "mAP_l": []
}

In [6]:
for i, cat_id in enumerate(coco_gt.getCatIds()): # 각 카테고리마다
    cat_name = coco_gt.loadCats(cat_id)[0]['name']
    coco_eval.params.catIds = [cat_id] 
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    metrics["category"].append(cat_name)
    metrics["mAP"].append(coco_eval.stats[0])  
    metrics["mAP_50"].append(coco_eval.stats[1])
    metrics["mAP_75"].append(coco_eval.stats[2]) 
    metrics["mAP_s"].append(coco_eval.stats[3])  
    metrics["mAP_m"].append(coco_eval.stats[4])  
    metrics["mAP_l"].append(coco_eval.stats[5]) 

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.42s).
Accumulating evaluation results...
DONE (t=0.71s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.517
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.689
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.557
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.039
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.322
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.628
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.599
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.618
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.192
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100

In [7]:
df_metrics = pd.DataFrame(metrics)
print(df_metrics)

        category       mAP    mAP_50    mAP_75     mAP_s     mAP_m     mAP_l
0  General trash  0.517200  0.688834  0.556724  0.039041  0.321839  0.628316
1          Paper  0.504926  0.746821  0.531792  0.041264  0.216573  0.588564
2     Paper pack  0.703972  0.851964  0.758210  0.000000  0.350602  0.812032
3          Metal  0.674579  0.828272  0.720200  0.095449  0.298399  0.801319
4          Glass  0.662780  0.861371  0.721062  0.004569  0.254990  0.739140
5        Plastic  0.583616  0.770979  0.631107  0.027037  0.284367  0.694945
6      Styrofoam  0.610247  0.830038  0.651207  0.000000  0.298994  0.659417
7    Plastic bag  0.680294  0.864945  0.747102  0.011256  0.262883  0.748628
8        Battery  0.824328  0.964793  0.897701 -1.000000  0.629401  0.847937
9       Clothing  0.721406  0.871457  0.784647  0.075743  0.156300  0.753100
